# Luminaries Twitter

Working attempt at updating code to only pull tweets after the last tweet id for a user or after a date

In [43]:
#!pip install tweepy

import pandas as pd
import numpy as np
import requests
import time
import pytz
import tweepy as tw
from tweepy import Cursor # Used to perform pagination
from tweepy import OAuthHandler # Used for authentication
from datetime import datetime, timedelta
from nltk.corpus import stopwords
import string

import re

from sklearn.decomposition import NMF, LatentDirichletAllocation

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud, STOPWORDS 
from collections import Counter


import matplotlib.pyplot as plt
%matplotlib inline
string.punctuation

stopWords = set(stopwords.words('english'))

In [2]:
TWITTER_KEY = 
TWITTER_SECRET = 
TWITTER_BEARER_TOKEN = 
ACCESS_TOKEN = 
ACCESS_TOKEN_SECRET = 

In [37]:
#Using the previous luminaries tweets file to get the last Tweet ID for each Luminary

client = tweepy.Client(bearer_token=TWITTER_BEARER_TOKEN)
auth = tweepy.OAuthHandler(TWITTER_KEY, TWITTER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
client2 = tweepy.API(auth, wait_on_rate_limit=True)

#auth = tw.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_token_secret)
#api = tw.API(auth, wait_on_rate_limit=True)

def get_tweepy_api():
    auth = tweepy.OAuthHandler(TWITTER_KEY, TWITTER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    return tweepy.API(auth, wait_on_rate_limit=True, retry_count=10, retry_delay=3)

api = get_tweepy_api()
tweet_result_df2 = pd.read_csv("full_tcc_lumin_tweets.csv", index_col=0)
#lumin_maxid = tweet_result_df2.groupby(by = 'screen_name').tweet_id.max().reset_index()
lumin_maxid = tweet_result_df2.groupby(by = ['screen_name', 'user_id']).tweet_id.max().reset_index()

In [ ]:
# Trying an approach that uses a query and date since, works for hashtag, 
# but how for Luminaries?
new_search = '#wildfires -filter:retweets'
date_since = "2022-02-15"
tweets = tw.Cursor(client2.search, 
                           q=new_search,
                           lang="en",
                           since=date_since).items(5)

In [38]:
# saving variable start and end dates, saving column names from previous export
lumin_maxid.columns=['handle', 'user_id','maxid']
s_time = '2022-01-01T00:00:00Z'
e_time = '2022-02-14T00:00:00Z'
col_names = list(tweet_result_df2.keys())

In [39]:
lumin_list=list(lumin_maxid['handle'][0:5])
userid_list = list(lumin_maxid['user_id'][0:5])
maxid_list = list(lumin_maxid['maxid'][0:5])

In [60]:
userid_list[0]

2474021983

In [40]:
tweet_results_df = {
    'tweet_text': [], 'tweet_id': [], 'created_at': [], 'tweet_yr': [], 'tweet_mo':[], 'tweet_mo_nbr':[],
    'tweet_day': [], 'tweet_hr': [],'tweet_day_of_week':[] ,'tweet_dow_nbr':[], 'tweet_date': [], 'is_extended_tweet': [],
    'is_retweet': [], 'is_quote_tweet': [], 'url_count': [], 'hashtag_count': [], 'lang': [],
    'user_id': [], 'screen_name': [], 'link_to_tweet': [], 'followers_count': [],
    'friends_count': [], 'user_created_at': [], 'user_statuses_count': [], 'user_tweets_per_day': [],
    'user_age_days': [], 'retweet_count': [], 'favorite_count': [], 'rechecked_time': []
}

col_names = list(tweet_results_df.keys())

In [41]:
# Function used in original Luminaries script, expects columns that are missing from the new scrips

month_dict = {'Jan':'1', 'Feb':'2', 'Mar':'3', 'Apr':'4', 'May':'5', 'Jun': '6', 
              'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct':'10', 'Nov':'11', 'Dec':'12' }
day_of_week = {'Sun':'1', 'Mon':'2', 'Tue':'3', 'Wed': '4', 'Thu': '5', 'Fri':'6', 'Sat':'7'}
def process_lumin_tweet(status,is_retweeted):
    
    
    try:
        status = status._json
    except AttributeError as error:
        pass
    
    #tweet_data = {}
    tweet_data = pd.DataFrame(columns = [col_names])
    #print(status)
    try:
        tweet_text = (status["extended_tweet"]["full_text"]).strip().replace('\n', ' ').replace('\r', '').replace(',', '').replace(u'’', u"'")
        is_extended=True
    except KeyError:
        tweet_text = (status["full_text"]).strip().replace('\n', ' ').replace('\r', '').replace(',', '').replace(u'’', u"'")
        is_extended=False
        
    tweet_id = str(status["id"])
    created_at = status["created_at"]
    tweet_yr = created_at[-4:-1]+created_at[-1]
    tweet_mo = created_at[4:7]
    tweet_mo_nbr = month_dict[tweet_mo]
    tweet_day = created_at[8:10]
    tweet_day_of_week = created_at[0:3]
    tweet_dow_nbr = day_of_week[tweet_day_of_week]
    tweet_hr = created_at[11:13]
    s = "-"
    seq = (tweet_yr, tweet_mo_nbr, tweet_day)
    tweet_date = s.join(seq)
    is_extended_tweet = is_extended
    is_retweet = is_retweeted
    is_quote_tweet = hasattr(status, "quoted_status")
    lang = status["lang"]
    
    if is_extended:
        url_count = len(status["extended_tweet"]["entities"]["urls"])
        hashtag_count = len(status["extended_tweet"]["entities"]["hashtags"])
    else:
        url_count = len(status["entities"]["urls"])
        hashtag_count = len(status["entities"]["hashtags"])
    
    user_json = status["user"]
    
    user_id = str(user_json["id"])
    screen_name = user_json["screen_name"]
    link_to_tweet = "https://twitter.com/{}/status/{}".format(tweet_data["screen_name"], status["id"])
    followers_count = user_json["followers_count"]
    friends_count = user_json["friends_count"]
    user_created_at = user_json["created_at"]
    user_statuses_count = user_json["statuses_count"]
    user_age_days = ((datetime.utcnow().replace(tzinfo=pytz.utc) - datetime.strptime(user_json['created_at'], "%a %b %d %H:%M:%S %z %Y")).days)
    try:
        tweets_per_day = user_json['statuses_count'] / user_age_days
    except ZeroDivisionError:
        tweets_per_day = 0
    user_tweets_per_day = tweets_per_day
    user_age_days = user_age_days
    
    retweet_count = status["retweet_count"]
    favorite_count = status["favorite_count"]
    rechecked_time = ((datetime.utcnow().replace(tzinfo=pytz.utc)))
    
    tweet_list = [tweet_text,
                  tweet_id,
                  created_at,
                  tweet_yr,
                  tweet_mo,
                  tweet_mo_nbr,
                  tweet_day,
                  tweet_hr,
                  tweet_day_of_week,
                  tweet_dow_nbr,
                  tweet_date,
                 is_extended_tweet,
                 is_retweet,
                 is_quote_tweet,
                 url_count,
                 hashtag_count,
                 lang,
                 user_id,
                 screen_name,
                 link_to_tweet,
                 followers_count,
                 friends_count,
                 user_created_at,
                 user_statuses_count,
                 user_tweets_per_day,
                 user_age_days,
                 retweet_count,
                 favorite_count,
                 rechecked_time]
    #print("tweet_list", tweet_list)

    return tweet_list


## Below are various attempts all work in process

In [58]:
# using Twitter API v2.0
# https://docs.tweepy.org/en/stable/client.html?highlight=user.timeline#tweepy.Client.get_users_tweets

tweet_fields = ['author_id','referenced_tweets.id','referenced_tweets.id.author_id',
                'entities.mentions.username','attachments.poll_ids','attachments.media_keys,in_reply_to_user_id',
                'geo.place_id']


all_tweets = []
for uzid, mxid, lumin in zip(userid_list, maxid_list, lumin_list):
    print(lumin)
    tweets = client.get_users_tweets(uzid, end_time=e_time, 
                                     since_id = mxid, 
                                     start_time = s_time, 
                                     expansions = tweet_fields,
                                     user_auth=False)
    print(tweets )
    all_tweets.extend(tweets)
        #print('N of tweets downloaded till now {}'.format(len(all_tweets)))
        


7CsHealth
[<Tweet id=1491681314113835009 text=L’Institut Bergonié, Exolis et Synapse Medicine s'allient dans le télésuivi des patients atteints de cancer via @rteston https://t.co/nhqmTS8U3F https://t.co/bS9zFEkW0l>, <Tweet id=1491085678301569027 text=Nouveau DMP : le service Mon Espace Santé interroge sur la sécurité des données https://t.co/xjfRiJVJAP https://t.co/g8hJB4VoBg>, <Tweet id=1491083917838934025 text=Des chercheurs développent un utérus artificiel géré par une IA https://t.co/U9x7aBZ8oF https://t.co/h7ABtnFnQ4>, <Tweet id=1490998350514200581 text=Le Pr Fabrice Denis, créateur primé de https://t.co/vRLLiar7OW : « nous ne sommes qu’aux balbutiements de la e-santé » https://t.co/3nTEhZOC9v https://t.co/KtsPt8vakA>, <Tweet id=1490988107369492485 text=Le CNOM publie un guide de la santé numérique via @rteston https://t.co/tX8UMGgxCB https://t.co/tOWp5vSbI7>, <Tweet id=1490988069939556355 text=Une nouvelle appli, truffée d’IA qui sélectionne les publications médicales qui vous c

In [59]:
all_tweets[0]

[<Tweet id=1491681314113835009 text=L’Institut Bergonié, Exolis et Synapse Medicine s'allient dans le télésuivi des patients atteints de cancer via @rteston https://t.co/nhqmTS8U3F https://t.co/bS9zFEkW0l>,
 <Tweet id=1491085678301569027 text=Nouveau DMP : le service Mon Espace Santé interroge sur la sécurité des données https://t.co/xjfRiJVJAP https://t.co/g8hJB4VoBg>,
 <Tweet id=1491083917838934025 text=Des chercheurs développent un utérus artificiel géré par une IA https://t.co/U9x7aBZ8oF https://t.co/h7ABtnFnQ4>,
 <Tweet id=1490998350514200581 text=Le Pr Fabrice Denis, créateur primé de https://t.co/vRLLiar7OW : « nous ne sommes qu’aux balbutiements de la e-santé » https://t.co/3nTEhZOC9v https://t.co/KtsPt8vakA>,
 <Tweet id=1490988107369492485 text=Le CNOM publie un guide de la santé numérique via @rteston https://t.co/tX8UMGgxCB https://t.co/tOWp5vSbI7>,
 <Tweet id=1490988069939556355 text=Une nouvelle appli, truffée d’IA qui sélectionne les publications médicales qui vous concer

In [35]:
all_tweets[0:3]
#users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in all_tweets]

AttributeError: 'list' object has no attribute 'user'

In [31]:
all_tweets2 = []
for uzid, mxid in zip(userid_list, maxid_list):
    try:
        tweets = get_tweets_from_user(uzid)
    except:
        pass
    
    all_tweets2.extend(tweets)
        #print('N of tweets downloaded till now {}'.format(len(all_tweets)))



In [32]:
temp_list=[]
for tweet in all_tweets:
    temp_list.append(process_lumin_tweet(tweet,is_retweeted=False))
df = pd.DataFrame(temp_list, columns=[col_names])
tweet_result_df = pd.concat([tweet_result_df, df])
print("N of tweets downloaded", len(all_tweets))

TypeError: list indices must be integers or slices, not str

In [28]:

def get_tweets_from_user(twitter_user_name, page_limit=16, count_tweet=200):
    """
    @params:
        - twitter_user_name: the twitter username of a user (company, etc.)
        - page_limit: the total number of pages (max=16)
        - count_tweet: maximum number to be retrieved from a page
        
    @return
        - all the tweets from the user twitter_user_name
    """
    
    for page in Cursor(api.user_timeline, 
                        screen_name=twitter_user_name, 
                        count=count_tweet).pages(page_limit):
        for tweet in page:
            parsed_tweet = {}
            parsed_tweet['date'] = tweet.created_at
            parsed_tweet['author'] = tweet.user.name
            parsed_tweet['twitter_name'] = tweet.user.screen_name
            parsed_tweet['text'] = tweet.text
            parsed_tweet['number_of_likes'] = tweet.favorite_count
            parsed_tweet['number_of_retweets'] = tweet.retweet_count

            all_tweets.append(parsed_tweet)

    # Create dataframe 
    df = pd.DataFrame(all_tweets)

    # Revome duplicates if there are any
    df = df.drop_duplicates( "text" , keep='first')
    return df


In [69]:
test = curl "https://api.twitter.com/2/users/2474021983/tweets?start_time=2022-01-01T00:00:00.000Z&end_time=2022-02-15T00:00:00.000Z&tweet.fields=id,created_at,text,author_id,attachments,geo,lang&user.fields=id,name,username,location&media.fields=url&place.fields=country_code" -H "Authorization: Bearer 

SyntaxError: invalid syntax (3573914286.py, line 1)

In [71]:
curl "https://api.twitter.com/2/users/2474021983/tweets" -H "Authorization: Bearer 

SyntaxError: invalid syntax (835869278.py, line 1)

In [73]:
import requests
r = requests.get("https://api.twitter.com/2/users/2474021983/tweets" -H "Authorization: Bearer 
r.json()


SyntaxError: invalid syntax (2288477334.py, line 2)

In [ ]:
curl "https://api.twitter.com/2/tweets?ids=1261326399320715264,1278347468690915330" \
  -H "Authorization: Bearer 
